# 06: Anomaly Explanation with Gemini 🧠

In this notebook, we bring our data to life. We use the **AnomalyExplainerAgent** (Day 5) to read the statistical outliers detected in Day 4 and generate human-readable business insights.

### 🎯 Goals
1.  **Load Anomalies:** Read the `anomaly_payload.json` generated by the Statistical Agent.
2.  **Initialize Agent:** Set up the `AnomalyExplainerAgent` with Gemini (Flash-Lite).
3.  **Generate Insights:** Run the agent on the top 5 highest-severity anomalies.
4.  **Review:** Compare the raw Z-Score data with the LLM's narrative.
5.  **Save Results:** Export the fully enriched dataset for the final dashboard.

### 🏗️ Components Used
* `agents.anomaly_llm_agent.AnomalyExplainerAgent`: The Gemini-powered explainer.
* `../outputs/anomalies/anomaly_payload.json`: The source data.

## 1: Imports

In [1]:
import sys
import os
import json
import pandas as pd
from dotenv import load_dotenv

# Add project root
project_root = os.path.abspath(os.path.join(os.path.dirname("__file__"), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

os.environ["OBSERVABILITY_DIR"] = os.path.join(project_root, "outputs", "observability")

# Load API Key
load_dotenv()

from agents.anomaly_llm_agent import AnomalyExplainerAgent

print("✅ Anomaly Explainer Agent Loaded")

c:\Users\YASH\anaconda3\envs\salesops\Lib\site-packages\google\cloud\aiplatform\models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


✅ Anomaly Explainer Agent Loaded


## 2: Load Data

In [2]:
INPUT_FILE = "../outputs/anomalies/anomaly_payload.json"

if not os.path.exists(INPUT_FILE):
    raise FileNotFoundError(f"Missing {INPUT_FILE}. Please run Notebook 05 first.")

with open(INPUT_FILE, "r") as f:
    data = json.load(f)

anomalies = data.get("top_anomalies", [])
print(f"Loaded {len(anomalies)} top anomalies from payload.")

# Preview the top 1
print("Top Anomaly ID:", anomalies[0]["anomaly_id"])
print("Score:", anomalies[0]["score"])

Loaded 50 top anomalies from payload.
Top Anomaly ID: iqr_Technology_2014-03-18_s53
Score: 53.08


## 3: Run Explanations

In [3]:
# Initialize Agent
agent = AnomalyExplainerAgent(model_name="gemini-2.5-flash-lite")
# Note: Using the latest Flash-Lite for speed and quality

# Pick top 3 for demo (to save time/quota)
target_anomalies = anomalies[:3]

# Run Batch Explanation
enriched_anomalies = agent.batch_explain(target_anomalies)

print("✅ Explanations Generated.")

✅ Explanations Generated.


## 4: Display Results

In [4]:
# Pretty Print the Structured Output
from IPython.display import display, Markdown

for rec in enriched_anomalies:
    # Handle errors gracefully
    if "error" in rec:
        display(
            Markdown(
                f"### ❌ Error for {rec.get('entity_id', 'Unknown')}: {rec['error']}"
            )
        )
        continue

    actions_bullet = "\n".join([f"    * {a}" for a in rec.get("suggested_actions", [])])
    meta = rec.get("meta", {})

    # Fallback title logic
    title_id = rec.get("entity_id") or rec.get("anomaly_id")

    md = f"""
### 🚨 Anomaly: {title_id} ({rec.get('level')})
**Severity:** {rec.get('score')} | **Confidence:** {rec.get('confidence')}

**📝 Short Explanation:**
{rec.get('explanation_short')}

**📄 Full Analysis:**
> {rec.get('explanation_full')}

**✅ Suggested Actions:**
{actions_bullet}

#### *Latency: {meta.get('latency_ms', 0)}ms | Ver: {meta.get('version', '1.0')}*
---
    """
    display(Markdown(md))


### 🚨 Anomaly: Technology (category)
**Severity:** 53.08 | **Confidence:** High

**📝 Short Explanation:**
Sales for Technology are significantly higher than expected, indicating a major outlier.

**📄 Full Analysis:**
> The current sales value of 24,739.75 is vastly exceeding the expected value of 585.69, as well as the third quartile (Q3) of 585.69. This magnitude of deviation suggests a substantial, likely one-time event, similar to past occurrences like a bulk order.

**✅ Suggested Actions:**
    * Investigate the source of the sales spike in Technology.
    * Verify if this spike is due to a legitimate large order or a data error.
    * Consult historical data for similar events to understand resolution.

#### *Latency: 2370ms | Ver: 1.1*
---
    


### 🚨 Anomaly: Office Supplies (category)
**Severity:** 29.98 | **Confidence:** High

**📝 Short Explanation:**
Office Supplies sales are significantly higher than expected, deviating from typical Q1 and Q3 ranges.

**📄 Full Analysis:**
> The current sales value of 6,929.22 for Office Supplies is an anomaly, as it is 29.98 times higher than the expected value of 260.77. This observed value falls far outside the interquartile range (IQR) of 222.42, which spans from 38.36 (Q1) to 260.77 (Q3). No direct historical parallels for office supplies anomalies are present in the memory bank.

**✅ Suggested Actions:**
    * Investigate the source of the unusually high sales for Office Supplies.
    * Verify the accuracy of the sales data for the current period.
    * Analyze recent customer orders and promotions related to Office Supplies.

#### *Latency: 1938ms | Ver: 1.1*
---
    


### 🚨 Anomaly: Office Supplies (category)
**Severity:** 22.21 | **Confidence:** High

**📝 Short Explanation:**
Office Supplies sales are significantly higher than expected, indicating a major anomaly.

**📄 Full Analysis:**
> The recorded sales of $8,263.37 for Office Supplies are an extreme outlier, being 22.21 times higher than the expected value of $399.59. This value far exceeds the upper quartile of $399.59 and indicates a substantial deviation from normal sales patterns, though no directly relevant historical events for Office Supplies anomalies were found in the memory bank.

**✅ Suggested Actions:**
    * Investigate the source of the unusually high sales for Office Supplies.
    * Verify transaction data for any errors or unusual bulk purchases.
    * Monitor Office Supplies sales for the next reporting period.

#### *Latency: 2233ms | Ver: 1.1*
---
    

## 5: Save Output

In [5]:
# Save these enriched records for the Reporting Agent (Day 6)
OUTPUT_FILE = "../outputs/anomalies/enriched_anomalies.json"

with open(OUTPUT_FILE, "w") as f:
    json.dump(enriched_anomalies, f, indent=2)

print(f"✅ Enriched anomalies saved to {OUTPUT_FILE}")

✅ Enriched anomalies saved to ../outputs/anomalies/enriched_anomalies.json


---
## ⏭️ Next Step: Closing the Loop (Action)

Success! We have successfully moved from **Data** → **Insight**.
* **Day 4:** We detected *what* happened (Statistical Anomalies).
* **Day 5:** We explained *why* it happened (AI Analysis with Gemini).
* **Output:** We have a rich dataset in `../outputs/anomalies/enriched_anomalies.json` containing scores, explanations, and suggested actions.

But a true SalesOps Agent isn't just an analyst—it is a **doer**. The final piece of the puzzle is taking these text-based suggestions and turning them into executable **actions**.

In the next notebook (Day 6), we will build the **Action Recommendation Agent**. This agent will:
1.  Read the enriched anomalies.
2.  Prioritize them based on severity and business impact.
3.  **Use Tools** (via a mock OpenAPI interface) to simulate real-world workflows, such as "Create Jira Ticket" or "Draft Email to Regional Manager".

👉 **Proceed to `notebooks/07_action_recommendation.ipynb`.**